Eventually it turned out to be easiest to download all docx and doc and use vba to convert to txt (Python cannot read .doc)

In [88]:
links = []
links.append(r"https://www.gov.uk/government/statistics/prison-population-2011")
links.append(r"https://www.gov.uk/government/statistics/prison-population-2012")
links.append(r"https://www.gov.uk/government/statistics/prison-population-figures-2014")
links.append(r"https://www.gov.uk/government/statistics/prison-population-figures")

In [89]:
from bs4 import BeautifulSoup
import requests

all_links = set()

for link in links:
    r = requests.get(link)
    soup = BeautifulSoup(r.content,"lxml")
    s = set([(a.text, a["href"]) for a in soup.findAll("a") if ".doc" in a["href"] and a.text != "" and "monthly" in a.text.lower()])
    all_links = all_links.union(s)

In [92]:
dls = ["https://www.gov.uk" + l[1] for l in list(all_links) if "docx" in l[1]]

In [5]:
import os 

In [6]:
# Download all .doc and save to folder for bulk conversion to .docx
# import urllib2
# dls = [l for l in list(all_links) if "docx" not in l[1]]
# dls
# for l in dls:
#     url = "https://www.gov.uk" + l[1]
#     head, tail = os.path.split(url)
#     my_file = urllib2.urlopen(url)
    
#     with open("downloads/" + tail,'wb') as output:
#       output.write(my_file.read())
    

In [7]:
# All the others
all_links = [l for l in all_links if "docx" in l[1]]

In [8]:
urls = iter(list(all_links))

In [9]:
from cStringIO import StringIO
import re
from docx import Document
import urllib2



In [55]:
def file_to_df(doc,my_file):
    re_pattern = r"\t\d{1,3}%$"

    paras = [p.text for p in doc.paragraphs if re.search(re_pattern, p.text) and p.text.count("\t") > 4]
    paras = [p.replace("\t**", "") for p in paras]

    f = "\n".join(paras)
    f2 = StringIO(f)
    import pandas as pd
    df = pd.read_csv(f2, sep = "\t", header=None)
    df.columns = ["prison_name","baseline_cna", "in_use_cna", "operational_capacity", "population", "perc_pop_to_used_cna", "perc_acc_available"]
    
    #Parse out the date here
    re_pattern = r"(\d{2}/\d{2}/\d{4})"
    
    f = " ".join([p.text for p in doc.paragraphs])
    date = re.search(re_pattern, f).group(1)   
    df["date"] = date
    df["source"] = my_file
    return df

In [56]:
# Create a tsv and then read in as a tsv
dfs = []
for tup in all_links:
    desc = tup[0]
    url = "https://www.gov.uk" + tup[1]
    document = Document()

    docx_file = urllib2.urlopen(url)
    docx_file = StringIO(docx_file.read())
    doc = Document(docx_file)
    dfs.append(file_to_df(doc,url))

 

In [57]:
# Now iterate through all the .docs which were converted in VBA
for i in os.listdir("vba_converted/"):
    if i.endswith(".docx"): 
        with open("vba_converted/"+i) as f:
            doc = Document(f)
            dfs.append(file_to_df(doc,i))
    else:
        continue

In [60]:
pd.options.display.max_colwidth = 10000
df_all = pd.concat(dfs)

In [63]:
gb = df_all.groupby("prison_name")

In [86]:
with open("prison-population-bulletin-march-2011.docx") as f:
    doc = Document(f)
#     dfs.append(file_to_df(doc,i))
#     re_pattern = r"\t\d{1,3}%$"

#     paras = [p.text for p in doc.paragraphs if re.search(re_pattern, p.text) and p.text.count("\t") > 4]
#     paras = [p.replace("\t**", "") for p in paras]

#     f = "\n".join(paras)
    